In [ ]:
import numpy as np
from src.levy_vs_montecarlo_demo import levy_fft_method
tauS_i = np.array([10, 100])  # Fast and slow sites
DeltaF_i = np.array([0.8, 0.2])  # 80% fast, 20% slow
rM_bar = 50  # Mean number of adsorption events
time_levy, peak_levy = levy_fft_method(tauS_i, DeltaF_i, rM_bar, n_points=2048)
import matplotlib.pyplot as plt
plt.plot(time_levy, peak_levy)

In [ ]:
from src.levy_vs_montecarlo_demo import simple_comparison
simple_comparison()

---

## Pasti 2005: Beyond Just the CF Formula

**Question:** If we only use the CF approach (not Monte Carlo), what does the Lévy process perspective add?

**Answer:** The Lévy perspective provides **implementation constraints and design principles** that go beyond just "use this formula":

### 1. **Model Validation via Infinite Divisibility**

**Constraint:** Not every function can be a valid CF for a Lévy process!

The Lévy-Khintchine theorem requires:
- φ(ω) must be **infinitely divisible**
- This means: φ(ω) = [φ(ω/n)]^n for all n

**Practical implication:**
If you try to fit a model with arbitrary parameters, you must check that the resulting CF is valid:

```python
def is_valid_levy_cf(cf_function, omega_test):
    """
    Test if a CF is infinitely divisible (necessary for Lévy process).
    """
    # For compound Poisson: φ(ω) = exp[λ(φ_X(ω) - 1)]
    # This form is ALWAYS infinitely divisible if φ_X is a valid CF
    
    # Check: φ_X must satisfy |φ_X(ω)| ≤ 1 for all ω
    phi_X = cf_function(omega_test)
    
    if np.any(np.abs(phi_X) > 1 + 1e-10):
        return False, "Jump CF has |φ| > 1 (invalid)"
    
    if not np.all(np.isfinite(phi_X)):
        return False, "CF contains inf/nan"
    
    return True, "Valid compound Poisson CF"
```

This prevents you from accidentally fitting nonsensical models!

### 2. **Moment Formulas Directly from Lévy Measure**

**Insight:** The Lévy measure ν(dτ) directly gives you moments WITHOUT computing the full PDF!

For compound Poisson with rate λ and jump distribution f_s(τ):
```
ν(dτ) = λ · f_s(τ) dτ
```

**Moment formulas:**
```python
# Mean (first moment)
mean = γ + ∫ τ ν(dτ)
     = t0 + λ · E[τ]
     = t0 + n · τ̄

# Variance (second cumulant)
variance = ∫ τ² ν(dτ)
         = λ · E[τ²]  
         = n · E[τ²]    # NOT n·Var[τ]!

# For exponential: E[τ²] = 2τ̄²
variance = n · 2τ̄²  = 2nτ̄²
```

**Practical use:**
- **Validate fits without computing PDF**: Check if fitted parameters give correct mean/variance
- **Initialize optimization**: Use moment matching for initial guess
- **Diagnostic**: If moments don't match, something is wrong

In [ ]:
# Demonstration: Compute moments WITHOUT FFT inversion
import numpy as np

def levy_moments_from_measure(sites_list, t0=0):
    """
    Compute mean and variance directly from Lévy measure.
    
    sites_list: [(n_i, tau_i), ...] for each site type
    
    Returns: (mean, variance)
    """
    mean = t0
    variance = 0
    
    for n_i, tau_i in sites_list:
        # Exponential distribution: E[τ] = tau_i, E[τ²] = 2*tau_i²
        mean += n_i * tau_i
        variance += n_i * 2 * tau_i**2
    
    return mean, variance

# Example: Multi-site model
sites = [
    (1000, 0.068),  # NS sites: n=1000, τ=0.068
    (10, 1.6),      # S sites: n=10, τ=1.6
]
t0 = 5.0

mean_theory, var_theory = levy_moments_from_measure(sites, t0)

print("Moments from Lévy measure (no PDF computation!):")
print(f"  Mean     = {mean_theory:.2f}")
print(f"  Variance = {var_theory:.2f}")
print(f"  Std Dev  = {np.sqrt(var_theory):.2f}")

# Verify: Lévy measure formulas match the distribution parameters
# For exponential distribution with n sites of mean τ:
#   E[X] = n·τ
#   Var[X] = n·2·τ²  (compound Poisson with exponential jumps)

print(f"\n✓ Lévy measure gives exact moments analytically!")
print(f"  (No FFT computation required - direct from distribution parameters)")
print(f"\nFor optimization:")
print(f"  • Use moments to validate parameters before expensive FFT")
print(f"  • Quickly check if parameter set is physically reasonable")
print(f"  • Detect parameter degeneracies (e.g., n↑τ↓ vs n↓τ↑)")

### 3. **Modular Composition Principle**

**Key Lévy insight:** Independent Lévy processes **add** in the exponent.

If you have multiple independent retention mechanisms:
```
Process 1: φ₁(ω) = exp[log φ₁(ω)]
Process 2: φ₂(ω) = exp[log φ₂(ω)]

Combined: φ_total(ω) = φ₁(ω) × φ₂(ω) 
                     = exp[log φ₁(ω) + log φ₂(ω)]
```

**Practical implementation:**

```python
class LevyRetentionModel:
    def __init__(self):
        self.processes = []  # List of (name, log_cf_func)
    
    def add_process(self, name, log_cf_function):
        """Add independent retention mechanism"""
        self.processes.append((name, log_cf_function))
    
    def characteristic_function(self, omega):
        """Combined CF"""
        log_phi_total = 0
        for name, log_cf in self.processes:
            log_phi_total += log_cf(omega)
        return np.exp(log_phi_total)
```

**Why this matters:**
- Add NS sites, S sites, aggregation separately
- Easy to turn on/off mechanisms during fitting
- Natural parameter organization

### 4. **Tail Behavior Prediction**

**Lévy process theory predicts asymptotic tail behavior** without computing full PDF!

For compound Poisson with jump distribution f_s(τ):
```
P(total time > t) ~ exp(-βt)  as t → ∞
```

where β is determined by the Lévy measure.

**Practical use:**
- **Outlier detection**: Know what tails should look like
- **Regularization**: Penalize models with unphysical tails
- **Data quality**: Check if experimental tails match theory

### 5. **Numerical Stability Guidance**

**Lévy perspective tells you where numerical problems will occur:**

1. **Small jumps (τ → 0):**
   - Lévy measure diverges: ν(dτ) ~ τ^(-1-α)dτ for α ∈ (0,1)
   - Compound Poisson is fine (finite activity)
   - But integration needs care if extending to general Lévy

2. **Large frequencies (ω → ∞):**
   - CF decays: |φ(ω)| → 0
   - FFT needs sufficient bandwidth
   - Lévy theory gives decay rate

3. **Parameter bounds:**
   - All n_i ≥ 0 (Lévy measure is positive)
   - All τ_i > 0 (jumps are positive - subordinator)
   - Sum of ΔF_i = 1 (probability)

**Implementation:**
```python
def validate_levy_parameters(n_values, tau_values, deltaF_values):
    """
    Check if parameters satisfy Lévy process constraints.
    """
    if np.any(n_values < 0):
        raise ValueError("Event counts must be non-negative")
    
    if np.any(tau_values <= 0):
        raise ValueError("Residence times must be positive (subordinator)")
    
    if not np.isclose(np.sum(deltaF_values), 1.0):
        raise ValueError("Probabilities must sum to 1")
    
    if np.any(deltaF_values < 0) or np.any(deltaF_values > 1):
        raise ValueError("Probabilities must be in [0,1]")
    
    return True
```

### 6. **Parameter Identifiability**

**Critical Lévy insight:** Different parameterizations can give same CF!

For discrete distribution {τᵢ, pᵢ} with mean events r̄M:
```
φ(ω) = exp[r̄M · Σ(e^(iωτᵢ) - 1) · pᵢ]
```

**Non-identifiable:**
- Can't distinguish (r̄M=100, τᵢ=1) from (r̄M=50, τᵢ=2) if only one component
- Need constraints or multiple components

**Lévy perspective solution:**
- **Fix r̄M** from external knowledge (e.g., column geometry)
- **Or fix one τᵢ** as reference timescale
- **Or use regularization** based on physical priors

This is why Pasti 2005 emphasizes:
```
r̄M = (mean event rate) × (column transit time)
```

**Practical:**
```python
def fit_with_fixed_rate(data, rM_bar_fixed):
    """
    Fit {τᵢ, ΔFᵢ} with fixed total rate.
    This removes degeneracy.
    """
    # Only optimize over residence times and fractions
    # Rate parameter is constrained
    ...
```

## Summary: Pasti 2005 Lévy Awareness for CF-Only Implementation

### What you get from Lévy process perspective (WITHOUT Monte Carlo):

| # | Insight | Practical Benefit for Optimization |
|---|---------|-----------------------------------|
| 1 | **Infinite divisibility** | Model validation - reject invalid CFs |
| 2 | **Moment formulas** | Fast diagnostics without PDF computation |
| 3 | **Modular composition** | Natural multi-mechanism parameterization |
| 4 | **Tail behavior** | Regularization and outlier detection |
| 5 | **Numerical stability** | Parameter constraints and bounds |
| 6 | **Identifiability** | Fix degeneracies, constrain search space |

### Concrete Implementation Recipe:

```python
class LevyAwareSDM:
    """
    Lévy-aware implementation for fast optimization.
    Uses CF + FFT (no Monte Carlo).
    """
    
    def __init__(self, mechanisms):
        """mechanisms: list of (name, n_events, tau_values, probs)"""
        self.mechanisms = mechanisms
        self._validate_parameters()  # Hint #5
    
    def _validate_parameters(self):
        """Ensure Lévy process constraints"""
        for name, n, taus, probs in self.mechanisms:
            assert n >= 0, "Event count must be non-negative"
            assert np.all(taus > 0), "Residence times must be positive"
            assert np.isclose(sum(probs), 1.0), "Probabilities must sum to 1"
    
    def moments(self):
        """Compute mean/variance WITHOUT FFT"""  # Hint #2
        mean = var = 0
        for name, n, taus, probs in self.mechanisms:
            tau_mean = np.dot(taus, probs)
            tau_2nd_moment = np.dot(taus**2, probs)
            mean += n * tau_mean
            var += n * tau_2nd_moment
        return mean, var
    
    def log_cf(self, omega):
        """Modular characteristic function"""  # Hint #3
        log_phi = 0
        for name, n, taus, probs in self.mechanisms:
            # Each mechanism adds independently
            for tau, p in zip(taus, probs):
                log_phi += n * p * (np.exp(1j * omega * tau) - 1)
        return log_phi
    
    def pdf(self, t_grid):
        """FFT inversion - for final evaluation only"""
        omega = np.fft.fftfreq(len(t_grid), t_grid[1]-t_grid[0]) * 2*np.pi
        cf = np.exp(self.log_cf(omega))
        pdf = np.fft.ifft(cf).real
        return np.maximum(pdf, 0) / np.sum(pdf) / (t_grid[1]-t_grid[0])
    
    def is_valid(self):
        """Check infinite divisibility"""  # Hint #1
        # For compound Poisson, always valid if parameters pass validation
        return True
    
    def objective_function(self, data):
        """
        Fast optimization objective.
        Uses moments (#2) instead of full PDF when possible!
        """
        # First check: moment matching (FAST)
        mean_model, var_model = self.moments()
        mean_data = np.mean(data)
        var_data = np.var(data)
        
        moment_error = ((mean_model - mean_data)**2 + 
                       (var_model - var_data)**2)
        
        # If moments are way off, don't bother with PDF
        if moment_error > threshold:
            return large_penalty
        
        # Otherwise compute full PDF (SLOW)
        pdf_model = self.pdf(t_grid)
        pdf_data = histogram(data)
        return chi_squared(pdf_model, pdf_data)
```

### Key Point:

**Pasti 2005's Lévy perspective gives you:**
- **Design patterns** (modular composition)
- **Constraints** (parameter validation)
- **Shortcuts** (moment formulas)
- **Diagnostics** (identifiability, tail behavior)

**Not just:** "Here's a CF formula"

This is what makes it **Lévy-aware**, even if you only use the CF approach!

## Detailed Explanation: Where is `LevyAwareSDM` Actually Lévy-Aware?

Let me annotate the code above to show **exactly where** Lévy theorems are being invoked:

---

### 1. **`moments()` Method** - Direct Lévy-Khintchine Application

```python
def moments(self):
    mean = var = 0
    for name, n, taus, probs in self.mechanisms:
        tau_mean = np.dot(taus, probs)
        tau_2nd_moment = np.dot(taus**2, probs)
        mean += n * tau_mean        # ← THEOREM: E[X] = γ + λ·E[τ]
        var += n * tau_2nd_moment   # ← THEOREM: Var[X] = σ² + λ·E[τ²]
```

**Where Lévy-aware:**
- **Doesn't derive** these formulas from chromatography
- **Directly applies** Lévy-Khintchine theorem for compound Poisson
- For Lévy measure ν(dτ) = Σ pᵢδ(τᵢ), moments are: ∫τ^k ν(dτ) = Σ pᵢ·τᵢ^k
- This is **proven by the theorem** - no need to verify!

**Chromatographic derivation would be:** "Sum over all molecular trajectories, weight by probabilities, take ensemble average..."

**Lévy-aware shortcut:** "Theorem says moments come directly from Lévy measure."

---

### 2. **`log_cf()` Method** - Lévy-Khintchine Formula

```python
def log_cf(self, omega):
    log_phi = 0
    for name, n, taus, probs in self.mechanisms:
        for tau, p in zip(taus, probs):
            log_phi += n * p * (np.exp(1j * omega * tau) - 1)  # ← THEOREM!
```

**Where Lévy-aware:**
- This is **exactly** the Lévy-Khintchine integral: λ∫[e^(iωτ) - 1]ν(dτ)
- For discrete measure ν = Σ pᵢδ(τᵢ), integral becomes sum
- The `(e^(iωτ) - 1)` term is **from the theorem**, not derived
- The `-1` is the **Lévy-Khintchine compensator** (ensures process starts at 0)

**Why the `-1` matters (Lévy theorem):**
Without it, you'd get `e^(n·Σ p·e^(iωτ))` which doesn't represent a process starting at X₀=0. The compensator `-1` subtracts the initial state, giving the **increment** distribution.

---

### 3. **Modular Composition** - Lévy-Itô Independence

```python
for name, n, taus, probs in self.mechanisms:
    log_phi += ...  # ← Each mechanism ADDS to log(CF)
```

**Where Lévy-aware:**
- **Lévy-Itô theorem:** Independent Lévy processes add in the exponent
- If X₁, X₂, X₃ are independent Lévy processes:
  ```
  log φ_total(ω) = log φ₁(ω) + log φ₂(ω) + log φ₃(ω)
  ```
- Loop over `mechanisms` = loop over independent Lévy components
- **No derivation needed** - theorem guarantees this works!

**Chromatographic approach:** Would need to prove independence, show convolutions multiply CFs, etc.

**Lévy-aware:** "Theorem says independent processes add. Done."

---

### 4. **Parameter Validation** - Infinite Divisibility

```python
def _validate_parameters(self):
    assert n >= 0, "Event count must be non-negative"
    assert np.all(taus > 0), "Residence times must be positive"
    assert np.isclose(sum(probs), 1.0), "Probabilities must sum to 1"
```

**Where Lévy-aware:**
- These constraints ensure the CF is **infinitely divisible**
- Lévy-Khintchine theorem **requires:**
  - λ ≥ 0 (rate parameter non-negative)
  - Support of ν on [0, ∞) for subordinators (taus > 0)
  - ν is a probability measure (probs sum to 1 after normalization)
  
**This validation isn't just "good practice"** - it's **theorem requirements**!

---

### 5. **Two-Step Optimization** - Moment Screening

```python
def objective_function(self, data):
    # First check: moment matching (FAST)
    mean_model, var_model = self.moments()  # ← No FFT!
    
    if moment_error > threshold:
        return large_penalty  # ← Skip expensive PDF
    
    # Otherwise compute full PDF (SLOW)
    pdf_model = self.pdf(t_grid)
```

**Where Lévy-aware:**
- Exploits that **moments are analytical** (from Lévy measure)
- PDF requires FFT (numerical), but moments are **exact**
- **Theorem-based speedup:** Screen 1000 parameter sets using moments, only compute PDF for promising candidates

**Without Lévy awareness:** Would compute PDF every time (1000× slower)

**With Lévy awareness:** "Theorem gives moments for free - use them first!"

---

## The Key Insight:

**Lévy-aware ≠ using a specific formula**

**Lévy-aware = Recognizing you're implementing a Lévy process and exploiting the theorems:**

| Code Line | Theorem Invoked | Benefit |
|-----------|----------------|---------|
| `mean += n * tau_mean` | Lévy-Khintchine | No derivation |
| `(e^(iωτ) - 1)` | Compensated Poisson | Correct CF structure |
| `log_phi +=` in loop | Lévy-Itô independence | Modular design |
| `assert n >= 0` | Infinite divisibility | Model validation |
| `moments()` before `pdf()` | Cumulant theorem | Optimization speedup |

This is **exactly** what we just refactored in `felinger1999_stochastic_dispersive.py`:
- Added `levy_triplet` property
- Added `validate_infinite_divisibility()`
- Added `moments_from_levy_measure()`
- Rewrote `variance()` to invoke Lévy-Itô directly

The `LevyAwareSDM` class follows the same pattern but for a multi-mechanism SDM model!

## Concrete Example: Lévy-Aware vs Non-Lévy-Aware

Let's see the difference in practice with your SDM implementation:

---

### ❌ **Non-Lévy-Aware Approach** (Derive from scratch)

```python
def sdm_variance_manual(npi, tpi, N0, t0):
    """Manual derivation from chromatography principles."""
    
    # Step 1: Derive variance from sorption events
    # "Each molecule undergoes Poisson(npi) adsorption events"
    # "Each event lasts exponential(tpi) time"
    # "Compound Poisson variance formula: λ·E[X²]"
    variance_kinetic = npi * 2 * tpi**2  # Had to derive this!
    
    # Step 2: Derive variance from Brownian dispersion
    # "Axial dispersion adds variance = 2D·t/u²"
    # "Effective time is t0·(1 + k')"
    k_prime = npi * tpi / t0
    variance_brownian = 1/(2*N0) * t0 * (1 + k_prime)**2  # Had to derive this!
    
    # Step 3: Show they add (prove independence)
    # "Need to prove adsorption and dispersion are independent..."
    total_variance = variance_kinetic + variance_brownian
    
    return total_variance
```

**Problems:**
- ❌ Must derive every formula from physical principles
- ❌ Must prove independence to add variances
- ❌ No validation that result is mathematically valid
- ❌ Hard to extend to new mechanisms

---

### ✅ **Lévy-Aware Approach** (Trust theorems)

```python
def sdm_variance_levy_aware(npi, tpi, N0, t0):
    """Theorem-based calculation using Lévy structure."""
    
    # Extract Lévy triplet (γ, σ², ν)
    gamma = t0  # Drift
    lambda_rate = npi  # Jump rate
    levy_measure_variance = 2 * tpi**2  # For exponential: Var[τ] = 2τ̄²
    
    k_prime = npi * tpi / t0
    sigma_squared = 1/(2*N0) * t0  # Base Brownian variance
    
    # Apply Lévy-Itô theorem: variances ADD for independent components
    variance_drift = 0  # Deterministic
    variance_brownian = sigma_squared * (1 + k_prime)**2  # Time-changed
    variance_poisson = lambda_rate * levy_measure_variance  # From ν
    
    # Theorem guarantees this:
    total_variance = variance_drift + variance_brownian + variance_poisson
    
    return total_variance
```

**Advantages:**
- ✅ **No derivation** - Lévy-Itô theorem says variances add
- ✅ **Validation built-in** - If (γ, σ², ν) valid, result is valid
- ✅ **Clear structure** - Three independent components explicit
- ✅ **Easy to extend** - Add new Lévy component, variance adds automatically

---

### The Difference in Your SDM Code

**Current implementation (sdm-examine.ipynb):**
```python
def dispersive_monopore_pdf(npi, tpi, N0, t0, ...):
    # Builds CF correctly, but doesn't expose Lévy structure
    cf = np.exp(iw * t0) * ...  # Drift hidden
         * np.exp(-w**2 / (2*N0))  # Brownian hidden
         * np.exp(npi * (levy_exp - 1))  # Poisson hidden
```

**Lévy-aware refactoring:**
```python
class LevySDM:
    @property
    def levy_triplet(self):
        return {
            'gamma': self.t0,
            'sigma_squared': 1/(2*self.N0),
            'lambda': self.npi,
            'levy_measure': ExponentialMeasure(self.tpi)
        }
    
    def variance(self):
        # Lévy-Itô: TRUST the theorem
        triplet = self.levy_triplet
        k_prime = triplet['lambda'] * self.tpi / triplet['gamma']
        
        return (
            0 +  # Drift (deterministic)
            triplet['sigma_squared'] * (1 + k_prime)**2 +  # Brownian
            triplet['lambda'] * triplet['levy_measure'].variance()  # Poisson
        )  # Theorem says these add - done!
```

This is the refactoring we just did to `felinger1999_stochastic_dispersive.py` - making the Lévy theorems **explicit** in the code structure!